Infiltrationsbestimmung nach Green und Ampt

Mit dem Green-Ampt Modell lässt sich die Infiltrationsrate in eine Bodensäule mit einem Querschnitt von 1 m² als exponentielle Funktion der Zeit darstellen.

$ Formel §

In [112]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, Button, HBox, VBox
from ipywidgets import interactive_output
from IPython.display import clear_output


Messdaten für die Infiltration [l/s] in einen 1 m² großen Bodenquerschnitt können als csv-Datei durch Eingabe des vollständigen Dateipfades in das nachstehende Eingabefenster eingelesen werden.

Im zweiten Eingabefenster ist der Spaltenname der Zeitdatenreihe einzufügen.

In [113]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from ipywidgets import interact, interactive, fixed, interact_manual, Button, HBox, VBox
from ipywidgets import interactive_output
from IPython.display import clear_output

class GreenAmptModel:
    def __init__(self):
        # Die Folgenden Listen werden Initialisiert, alle mit 'self' markierten Variablem, listen etc. sind nicht nur innerhalb einer Funktion, sondern in der gesammten Klasse definiert
        self.reference_lines = []
        self.ref_line_params = []
        self.short_time_ref = []
        self.short_time_params = []
        self.infiltration_rate_ref = []
        self.infiltration_ref = []


    def t_for_It(self, I_t, d_theta, d_h, Ks=0.3):
            return (I_t - d_theta * d_h * np.log(1+ (I_t/(d_theta*d_h))))/Ks


    #def short_time_approximation(self, d_theta, d_h, K, t):
    """Short time approximation for cumulative infiltration
        Parameters
        ----------
        d_theta : float
            Change in volumetric water content
        d_h : float
            Change in pressure head
        K : float
            Hydraulic conductivity
        t : array_like
            Time
        Returns
        -------
        i_t : array
            Cumulative infiltration"""

     #   return np.sqrt((d_theta*d_h*K)/(2*t))

    def long_time_approximation(self, K, x):
        """Long time approximation for cumulative infiltration
        Parameters
        ----------
        K : float
            Hydraulic conductivity
        t : array_like
            Time
        Returns
        -------
        K : float
            Hydraulic conductivity"""

        it_long = np.array([K]*len(x))

        return it_long

    def d_h(self, h0=0, hf=-60):
        return h0-hf

    def d_theta(self, theta_0=0.44, theta_i=0.1):
        return theta_0 - theta_i
    
    def infiltration_rate(self, new_t, t, I_t):
        y2 = np.interp(new_t, t, I_t)
        i_t = np.diff(y2)
        #self.i_t = i_t
        return i_t
    
    # Um vom Benutzer, als "String", eingelesene Daten zu Plotten werden diese in eine "List" konvertiert
    def read_datastring(self, y_data_string='', x_data_string=''):
        if y_data_string or x_data_string != '':
            y_data = y_data_string.split(',')
            x_data = y_data_string.split(',')
            return y_data, x_data

    def plot_scenario(self, hf_h0=(-60,0), thetai_theta0=(0.2,0.44), Ks=4, i_data_string='',ti_data_string='', I_data_string='', tI_data_string='' ):

        # Speichern der über "Widgets" eingelesenen Werte in Variablen
        h0 = hf_h0[1]
        hf = hf_h0[0]
        theta_i = thetai_theta0[0]
        theta_0 = thetai_theta0[1]
        delta_theta = self.d_theta(theta_0, theta_i)
        delta_h = self.d_h(h0, hf)
        
        # Generieren der Daten 
        I_t = np.arange(0.1, 260, 0.1)                                                         # Array für die Infiltration
        t = self.t_for_It(I_t, delta_theta, delta_h, Ks)                                       # Array für die Zeit (der Infiltration)
        #np.savetxt('time.txt', t, delimiter=',')
        self.new_t = np.arange(round(t[0], 1), round(t[-1], 1), 0.01)                          # Array für die Zeit auf 1 Dezimalstelle gerundet
        self.i_t = self.infiltration_rate(self.new_t, t, I_t)                                                           # Array für die Infiltrationsrate
        i_t_long = self.long_time_approximation(Ks, self.new_t)                                # Array für die Langzeitannäherung ~ Ks

        # Erstellen der Abbildng
        fig, (ax1, ax2) = plt.subplots(2, figsize=(12,12))

        # Für den Button: Plotten der Referenzlinien, der gespeicherten Graphen
        for ref, params in zip(self.infiltration_ref, self.ref_line_params):
            ref_x, ref_y = ref
            ax1.plot(ref_x, ref_y, linestyle='-.', alpha=0.6, label='hf:{} h0:{} theta_i:{} theta_0:{} Ks:{}'.format(params[0], params[1], params[2], params[3], params[4]), linewidth=2.5)
        
        for ref, params in zip(self.infiltration_rate_ref, self.ref_line_params):
            ref_x, ref_y = ref
            ax2.plot(ref_x, ref_y, linestyle='-.', alpha=0.6, label='hf:{} h0:{} theta_i:{} theta_0:{} Ks:{}'.format(params[0], params[1], params[2], params[3], params[4]), linewidth=2.5)
        
        # Plotten der Daten
        ax1.plot(t, I_t, color='black', label='I(t)', linewidth=1)
        
        ax2.plot(self.new_t[1:-5], self.i_t[0:-5]*100, color='black', label='i(t)', linewidth=1)
        ax2.plot(self.new_t, i_t_long, 'red', linestyle='--', label='Long time approximation ~ Ks [mm/min]')

        # Formatierung
        ax1.set_ylim(0.1, max(I_t))
        ax1.set_xlim(0, 185)
        ax1.set_xlabel('Time [min]', fontsize=10)
        ax1.set_ylabel('infiltration rate [mm/min]', fontsize=10)

        ax2.set_ylim(0.1, 150)
        ax2.set_xlim(0, 185)
        ax2.set_yscale('log')
        ax2.set_xlabel('Time [min]', fontsize=10)
        ax2.set_ylabel('infiltration rate [mm/min]', fontsize=10)

        #ax1.set_yscale('log')

        # Platzieren der Legende
        ax1.legend(loc='best', bbox_to_anchor=(0, 0))
        ax2.legend(loc='best', bbox_to_anchor=(0, 0))

        # Plotten von zusätzlichen, vom Benutzer eingegebenen Messdaten
        ## für die Infiltrationsrate
        if i_data_string or ti_data_string != '':                                                 # Wurden Messdaten übergeben? Wenn, ja:
            i_data, ti_data = self.read_datastring(i_data_string, ti_data_string)                 # Umwandeln von str -> list
            ax2.plot(ti_data, i_data, color='blue', linestyle='', marker='+')                     # Plotten

        ## für die Infiltration
        if I_data_string or tI_data_string != '':                                                 # Wurden Messdaten übergeben? Wenn, ja:
            I_data, tI_data = self.read_datastring(I_data_string, tI_data_string)                 # Umwandeln von str -> list
            ax1.plot(tI_data, I_data, color='blue', linestyle='', marker='+')                     # Plotten

        plt.show()

    def on_button_click(self, button):
        # Speichern der über "Widgets" eingelesenen Werte in Variablen
        h0_val = self.hf_h0.value[1]
        hf_val = self.hf_h0.value[0]
        theta_i_val = self.thetai_theta0.value[0]
        theta_0_val = self.thetai_theta0.value[1]
        Ks_val = self.Ks_slider.value

        delta_theta = (self.d_theta(theta_0_val, theta_i_val))
        delta_h = self.d_h(h0_val, hf_val)

        I_t = np.arange(0, 260, 0.1)
        t = self.t_for_It(I_t, delta_theta, delta_h, Ks_val)
        self.new_t = np.arange(round(t[0], 1), round(t[-1], 1), 0.01)
        #i_t_short = self.short_time_approximation(delta_theta, delta_h, Ks_val, self.new_t)
        i_t = self.infiltration_rate(self.new_t, t, I_t)
        print('t', len(t), 'i_t', len(i_t))

        # Store the current short time approximation data as a reference line
        self.reference_lines.append((t, I_t))
        self.ref_line_params.append((h0_val, hf_val, theta_0_val, theta_i_val, Ks_val))
        self.reference_lines.append((self.new_t, i_t))
        self.ref_line_params.append((h0_val, hf_val, theta_0_val, theta_i_val, Ks_val))
        #self.short_time_ref.append((self.new_t, i_t))
        #self.short_time_params.append((h0_val, hf_val, theta_0_val, theta_i_val, Ks_val))

        #it = self.i_t(self.new_t, t, I_t)
        self.infiltration_ref.append((t, I_t))
        self.infiltration_rate_ref.append((self.new_t[1:-5], self.i_t[:-5]*100))

        clear_output(wait=True)
        #display(self.interactive_plot)
        #display(button)

        #display(widgets.VBox(children=[tab, button]))


    def create_interactive_plot(self):
        # Create the button
        button = Button(description="Fixate Short time approximation")
        button.on_click(self.on_button_click)

        # Create sliders for the parameters
        self.hf_h0 = widgets.FloatRangeSlider(value=[-600, 0], min=-1000, max=100, step=0.1, description=('hf - h0 [mm]'))
        self.thetai_theta0 = widgets.FloatRangeSlider(value=[0.1, 0.4], min=0, max=0.5, step=0.01, description=('theta i - theta 0 []'))
        self.Ks_slider = widgets.FloatSlider(value=0.5, min=0.01, max=5, step=0.01, description='Ks [mm/min]' )

        # Sliders for reading in data 
        self.I_data_string = widgets.Textarea(placeholder='Daten Bitte mit . als Dezimaltrennzeichen und , als Trennzeichen eingeben', description='Infiltrationsmessdaten')
        self.tI_data_string = widgets.Textarea(placeholder='Daten Bitte mit . als Dezimaltrennzeichen und , als Trennzeichen eingeben', description='Zeitdaten für I(t)')

        self.i_data_string = widgets.Textarea(placeholder='Daten Bitte mit . als Dezimaltrennzeichen und , als Trennzeichen eingeben', description='Infiltrationsrate')
        self.ti_data_string = widgets.Textarea(placeholder='Daten Bitte mit . als Dezimaltrennzeichen und , als Trennzeichen eingeben', description='Zeitdaten für i(t)')

        tab1 = VBox(children=[self.hf_h0, self.thetai_theta0, self.Ks_slider])
        tab2 = VBox(children=[self.I_data_string, self.tI_data_string])
        tab3 = VBox(children=[self.i_data_string, self.ti_data_string])

        tab = widgets.Tab(children=[tab1, tab2, tab3])
        tab.set_title(0, 'Variablen')
        tab.set_title(1, 'Messdaten I(t)')
        tab.set_title(2, 'Messdaten i(t)')

        #interactive(self.plot_scenario, hf_h0= self.hf_h0, thetai_theta0 = self.thetai_theta0, Ks=self.Ks_slider, i_data_string=self.i_data_string, ti_data_string=self.ti_data_string, I_data_string=self.I_data_string, tI_data_string=self.tI_data_string)
        widgets.VBox(children=[tab, button])
        #out = self.interactive_plot = interactive_output(self.plot_scenario, {hf_h0 = self.hf_h0, thetai_theta0 = self.thetai_theta0, Ks=self.Ks_slider, i_data_string=self.i_data_string, ti_data_string=self.ti_data_string, I_data_string=self.I_data_string, tI_data_string=self.tI_data_string})
        #display(box, out)
        #self.plot_scenario(self.hf_h0, self.thetai_theta0, self.Ks_slider, self.i_data_string, self.ti_data_string, self.I_data_string, self.tI_data_string)
        

        # Display the interactive plot with the sliders and button
        self.interactive_plot = interactive(self.plot_scenario, hf_h0 = self.hf_h0, thetai_theta0 = self.thetai_theta0, Ks=self.Ks_slider, i_data_string=self.i_data_string, ti_data_string=self.ti_data_string, I_data_string=self.I_data_string, tI_data_string=self.tI_data_string)

        display(self.interactive_plot)        
        #display(button)



In [114]:
GreenAmptModel().create_interactive_plot()


interactive(children=(FloatRangeSlider(value=(-600.0, 0.0), description='hf - h0 [mm]', min=-1000.0), FloatRan…

In [115]:

def read_csv(data_path, col_name_inf, col_name_t):

    if data_path and col_name_inf and col_name_t != '':
        data = pd.read_csv(data_path, parse_dates=True, sep=';')
        infiltration_data = data[col_name_inf].values
        timesteps_data = data[col_name_t].values

        if len(infiltration_data) == len(timesteps_data):
            return infiltration_data, timesteps_data
        else:
            print('!Fehler!: Die Infiltrationsdaten haben nicht die selbe Länge, wie die Zeitdaten')

    else:
        print('!Fehler! :Es sind nicht alle Felder korekt ausgefüllt, es werden keine Daten eingelesen')


def plot_Infiltration(data_path, col_name_inf, col_name_t):

    infiltration_data, timesteps_data = read_csv(data_path, col_name_inf, col_name_t)

    fig, ax = plt.subplots(figsize=(10, 6))

    ax.plot(timesteps_data, infiltration_data, marker='x', color='orange', linestyle='', label='Messdaten Infiltrationsversuch')
    xticks = np.arange(0,30,10)
    ax.set_xticks(xticks)

    #ax.set_ylim(0.1, max(infiltration_data)+1)
    #ax.set_xlim(-5, max(timesteps_data))
    plt.yscale('log')
    ax.set_xlabel('Time [h]', fontsize=12)
    ax.set_ylabel('infiltration rate [cm/h]', fontsize=12)

    # Move the legend outside of the plot
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

    plt.show()
    


interact(plot_Infiltration,
         data_path = widgets.Text(placeholder='Hier Dateipfad einfügen', description='Dateipfad: ', disabled=False),
         col_name_t = widgets.Text(placeholder='Hier Spaltenname für Zeitdaten einfügen', description='Zeitdaten: ', disabled=False),
         col_name_inf = widgets.Text(placeholder='Hier Spaltenname für Infiltraton einfügen', description='Infiltrationsdaten: ', disabled=False)
        )


#cum + sch
#mm/min
#eingabe für f1. plt und F 2. plt und t


interactive(children=(Text(value='', description='Dateipfad: ', placeholder='Hier Dateipfad einfügen'), Text(v…

<function __main__.plot_Infiltration(data_path, col_name_inf, col_name_t)>